# 0. Preparation


In [ ]:
from pathlib import Path
import numpy as np
import matplotlib.pyplot as plt
import cv2

from skimage.feature import hog
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report

import tensorflow as tf
from tensorflow.keras import layers, models

from google.colab import drive

drive.mount("/content/drive")

# 1. Dataset Loading


In [ ]:
def load_dataset(img_dir):
    p = Path(img_dir)
    img_list = []
    for folder in p.glob("*"):
        label = folder.name
        for file in folder.glob("*.jpg"):
            img = cv2.imread(str(file))
            img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
            img_list.append((img, label))
    return img_list


train_dir = "/content/drive/MyDrive/FolderAnda/images/training/"
test_dir = "/content/drive/MyDrive/FolderAnda/images/test"

train_img = load_dataset(train_dir)
test_img = load_dataset(test_dir)

print(f"Jumlah data training: {len(train_img)}")
print(f"Jumlah data testing: {len(test_img)}")

# 2. Data Preprocessing


In [ ]:
def resize_image(img, size=(256, 256)):
    return cv2.resize(img, size)


def label_encoder(label):
    return 1 if label == "day" else 0


def preprocess(img_list):
    X = []
    y = []
    for img, label in img_list:
        img_std = resize_image(img)
        X.append(img_std)
        y.append(label_encoder(label))
    return X, y


X_train_img, y_train = preprocess(train_img)
X_test_img, y_test = preprocess(test_img)

# 3. Feature Extraction


In [ ]:
def extract_hog(X_imgs):
    feats = []
    for img in X_imgs:
        gray = cv2.cvtColor(img, cv2.COLOR_RGB2GRAY)
        hog_feat = hog(
            gray,
            orientations=9,
            pixels_per_cell=(8, 8),
            cells_per_block=(2, 2),
            block_norm="L2-Hys",
            visualize=False,
            feature_vector=True,
        )
        feats.append(hog_feat)
    return np.array(feats)


X_train_feat = extract_hog(X_train_img)
X_test_feat = extract_hog(X_test_img)

# 4. Normalization


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_feat)
X_test_scaled = scaler.transform(X_test_feat)

# 5. Label Conversion


In [ ]:
y_train = np.array(y_train)
y_val = np.array(y_val)
y_test = np.array(y_test)

# 6. Modelling


In [ ]:
input_dim = X_train.shape[1]

model = models.Sequential(
    [
        layers.Input(shape=(input_dim,)),
        layers.Dense(128, activation="relu"),
        layers.Dense(64, activation="relu"),
        layers.Dense(1, activation="sigmoid"),
    ]
)

model.compile(optimizer="adam", loss="binary_crossentropy", metrics=["accuracy"])

model.summary()

# 7. Model Training


In [ ]:
history = model.fit(
    X_train, y_train, epochs=20, batch_size=32, validation_data=(X_val, y_val)
)

# 8. Model Evaluation


In [ ]:
test_loss, test_acc = model.evaluate(X_test_scaled, y_test)
print("Akurasi Test:", test_acc)

In [ ]:
y_pred_prob = model.predict(X_test_scaled)
y_pred = (y_pred_prob > 0.5).astype(int)

print(classification_report(y_test, y_pred))
print(confusion_matrix(y_test, y_pred))

# Visualize training history
plt.plot(history.history["accuracy"], label="Train Acc")
plt.plot(history.history["val_accuracy"], label="Val Acc")
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.legend()
plt.show()

# 9. Saved the Model


In [ ]:
import pickle

model.save("day_night_model.h5")
print("✅ Model berhasil disimpan sebagai: day_night_model.h5")

with open("scaler.pkl", "wb") as f:
    pickle.dump(scaler, f)
print("✅ Scaler berhasil disimpan sebagai: scaler.pkl")